In [1]:
import collections
import datetime
import json
import os
import pprint
import random

import ee
import numpy as np
import pandas as pd
import openet.core

from IPython.display import Image, display
import ipyplot


In [2]:
ee.Initialize(
    project='ee-cmorton',
    opt_url='https://earthengine-highvolume.googleapis.com'
)

In [3]:
# etf_coll_id = 'projects/openet/assets/ssebop/conus/gridmet/landsat/c02'
etf_coll_id = 'projects/usgs-gee-nhm-ssebop/assets/ssebop/landsat/c02'
# etf_coll_id = 'projects/openet/assets/intercomparison/ssebop/landsat/c02/v0p2p6'
band_name = 'et_fraction'

wrs2_skip_list = [
    'p010r027', 
    # Extra Canada scenes (not sure why they are in WRS2 list)
    'p035r025', 'p034r025', 'p033r025', 'p032r025', 'p031r025', 'p030r025', 'p029r025', 'p028r025', 'p027r025', 
]

land_mask = ee.Image('projects/openet/assets/features/water_mask').Not()
# Apply the NLCD/NALCMS water mask (anywhere it is water, set the ocean mask 
land_mask = land_mask.where(ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS").unmask(18).eq(18), 0)
# land_mask = land_mask.And(ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS").unmask(18).neq(18))
# # land_mask = ee.Image('projects/openet/assets/meteorology/conus404/ancillary/land_mask')

stats_ws = os.path.join(os.getcwd(), 'stats')
moran_ws = os.path.join(os.getcwd(), 'stats_moran')
#if not os.path.isdir(stats_ws):
#    os.makedirs(stats_ws)

# image_size = 1024
# image_size = 700

rgb_bands = {
    'LT04': ['SR_B3', 'SR_B2', 'SR_B1'],
    'LT05': ['SR_B3', 'SR_B2', 'SR_B1'],
    'LE07': ['SR_B3', 'SR_B2', 'SR_B1'],
    'LC08': ['SR_B4', 'SR_B3', 'SR_B2'],
    'LC09': ['SR_B4', 'SR_B3', 'SR_B2'],
}

ocean_wrs2_list = [
    'p048r027', 'p047r031', 'p047r030', 'p046r033', 'p045r034', 
    'p044r035', 'p043r036', 'p041r037', 'p040r038', 
    'p024r040', 'p024r027', 'p023r040', 'p023r027', 'p020r029',
    'p017r041', 'p016r038', 'p015r040', 'p015r037', 
    'p013r033', 'p012r032', 'p011r031', 'p011r030', 
]

# 0 - white, 1 - no fill (green), 2 - shadow (dark blue), 3 - snow (light blue), 4 - cloud (light gray), 5 - water (purple), 6 - ocean mask
fmask_palette = "ffffff, 9effa1, blue, 00aff2, dddddd, purple, bfbfbf"
fmask_max = 6

ocean_wrs2_list = [
    'p048r027', 
    'p047r031', 
    'p047r030', 
    'p047r029', 
    'p046r033', 
    'p045r034', 
    'p044r035', 
    'p043r036', 
    'p041r037', 
    'p040r038', 
    'p025r040', 
    'p024r040', 
    'p024r027', 
    'p023r040', 
    'p023r027', 
    'p022r040', 
    'p021r040',
    'p020r029',
    'p017r041', 
    'p016r038', 
    'p015r040', 
    'p015r037', 
    'p013r033', 
    'p012r032', 
    'p011r031', 
    'p011r030', 
]

In [4]:
# Get the list of WRS2 tiles from the SSEBop collection
wrs2_list = sorted(
    ee.ImageCollection(etf_coll_id).filterDate('2020-01-01', '2024-01-01')
    .aggregate_histogram('wrs2_tile').keys().getInfo(),
    reverse=True
)
wrs2_list = set(wrs2_list + ['p018r028'])
# pprint.pprint(wrs2_list)

In [5]:
# # Intercomparison sites and dates

# sites_csv = '/Users/Charles.Morton@dri.edu/Projects/openet-tools/intercomparison/master_flux_station_list.csv'
# sites_df = pd.read_csv(sites_csv)

# interp_days = 32
# site_keep_list = []
# wrs2_delimiter = ';'

# # Hardcoding the sites CSV field names for now
# start_field = 'START_DATE'
# end_field = 'END_DATE'
# site_field = 'SITE_ID'
# lat_field = 'LATITUDE'
# lon_field = 'LONGITUDE'
# wrs2_field = 'WRS2_TILES'

# from dateutil.relativedelta import relativedelta

# # Group the date ranges by WRS2 tile
# # print(f'\nGrouping overlapping dates')
# wrs2_dates = collections.defaultdict(list)
# wrs2_sites = collections.defaultdict(list)
# for (site_i, site) in sites_df.iterrows():
#     # print(site_i)
#     # print(site)
#     if site['RANDOM_SELECTION'] not in [0, 1]:
#         # print('  Unsupported RANDOM_SELECTION value')
#         input('ENTER')
#         continue
#     # if site['RANDOM_SELECTION'] != 1:
#     #     continue
#     if site_keep_list and site.loc[site_field] not in site_keep_list:
#         # print('  Site not in keep list - skipping')
#         continue

#     # Include all sites in INI file, even those outside the date range
#     for wrs2 in site.loc[wrs2_field].split(wrs2_delimiter):
#         wrs2_sites[wrs2.strip()].append([
#             round(site.loc[lon_field], 6), round(site.loc[lat_field], 6)
#         ])

#     start_dt = datetime.datetime.strptime(site.loc[start_field], '%Y-%m-%d')
#     end_dt = datetime.datetime.strptime(site.loc[end_field], '%Y-%m-%d')
#     # print(f'  Start Date: {start_dt.strftime("%Y-%m-%d")}')
#     # print(f'  End Date:   {end_dt.strftime("%Y-%m-%d")}')

#     # If start/end dates are within N gap days of the start/end of the month
#     #   consider it a "full" month
#     gap_days = 5
#     # print('  Snapping start date to month')
#     month_start_dt = datetime.datetime(start_dt.year, start_dt.month, 1)
#     if (start_dt - month_start_dt).days <= gap_days:
#         # print('    full month')
#         start_dt = month_start_dt
#     else:
#         # print('    not full month')
#         start_dt = month_start_dt + relativedelta(months=1)

#     # print('  Snapping end date to month')
#     month_end_dt = end_dt + relativedelta(months=1)
#     month_end_dt = datetime.datetime(month_end_dt.year, month_end_dt.month, 1)
#     month_end_dt = month_end_dt - relativedelta(days=1)
#     if (month_end_dt - end_dt).days <= gap_days:
#         # print('    full month')
#         end_dt = month_end_dt
#     else:
#         # print('    not full month')
#         end_dt = (month_end_dt + relativedelta(days=1) -
#                   relativedelta(months=1) - relativedelta(days=1))
#     # print(f'  Start Date: {start_dt.strftime("%Y-%m-%d")}')
#     # print(f'  End Date:   {end_dt.strftime("%Y-%m-%d")}')

#     if interp_days > 0:
#         # Buffer the date ranges by the interpolate days value if set
#         # print('  Buffering start/end dates')
#         start_dt = start_dt - datetime.timedelta(days=interp_days)
#         end_dt = end_dt + datetime.timedelta(days=interp_days)
#         # print(f'  Start Date: {start_dt.strftime("%Y-%m-%d")}')
#         # print(f'  End Date:   {end_dt.strftime("%Y-%m-%d")}')

#     # CM - Changing conditionals to get single date ranges to work
#     # if end_dt <= start_dt or start_dt >= end_dt:
#     if end_dt < start_dt or start_dt > end_dt:
#         # print(f'  Start: {start_dt.strftime("%Y-%m-%d")}')
#         # print(f'  End:   {end_dt.strftime("%Y-%m-%d")}')
#         # print('  Date range outside min/max, skipping')
#         continue
#     else:
#         # print(f'  Start: {start_dt.strftime("%Y-%m-%d")}')
#         # print(f'  End:   {end_dt.strftime("%Y-%m-%d")}')
#         pass

#     for wrs2 in site.loc[wrs2_field].split(wrs2_delimiter):
#         wrs2_dates[wrs2.strip()].append([start_dt, end_dt])

# # pprint.pprint(wrs2_dates)

# # Merge the date ranges that overlap
# print(f'\nMerging overlapping dates')
# merged_dates = {}
# for wrs2, dates in sorted(wrs2_dates.items()):
#     # print(f'  {wrs2}')
#     # pprint.pprint(sorted(dates))

#     # Push the first interval on to the stack
#     merged_dates[wrs2] = [sorted(dates)[0]]

#     # Only check for overlapping ranges if there is more than 1 range
#     if len(dates) == 1:
#         continue

#     for d in sorted(dates)[1:]:
#         # If the current date range doesn't overlap, add it to the stack
#         if d[0] > merged_dates[wrs2][-1][1]:
#             merged_dates[wrs2].append(d)
#         # If the ranges overlap and the end date is later,
#         #   update the end time of the stack value
#         elif ((d[0] <= merged_dates[wrs2][-1][1]) and
#               (d[1] > merged_dates[wrs2][-1][1])):
#             merged_dates[wrs2][-1][1] = d[1]

# # pprint.pprint(merged_dates)

# # # CGM - Splitting by year for DisALEXI is not needed if the NLCD
# # #   is set to the image collection instead of the image
# # # For DisALEXI split the date ranges by year after merging
# # # For other models, index by the first year in the range
# # # This may be functionality we will want for other models later
# # year_dates = collections.defaultdict(dict)
# # # if model in ['DISALEXI_TAIR_10K', 'DISALEXI_TAIR_1K', 'DISALEXI', 'DISALEXI_TAIR_DIRECT']:
# # #     for wrs2, dates in merged_dates.items():
# # #         # split_dates = {}
# # #         for date_i, date in enumerate(dates):
# # #             for year in range(date[0].year, date[1].year+1):
# # #                 year_date = [
# # #                     max(date[0], datetime.datetime(year, 1, 1)),
# # #                     min(date[1], datetime.datetime(year, 12, 31)),
# # #                 ]
# # #                 try:
# # #                     year_dates[wrs2][year].append(year_date)
# # #                 except:
# # #                     year_dates[wrs2][year] = [year_date]
# # # else:
# # for wrs2, dates in merged_dates.items():
# #     year_dates[wrs2][dates[0][0].year] = dates

# # pprint.pprint(year_dates)


In [6]:
def fmask(landsat_img):
    # Add the fmask image on top of the true color image
    qa_img = landsat_img.select('QA_PIXEL')
    fill_mask = qa_img.bitwiseAnd(1).neq(0)                  # bits: 0
    dilate_mask = qa_img.rightShift(1).bitwiseAnd(1).neq(0)  # bits: 1
    cirrus_mask = qa_img.rightShift(2).bitwiseAnd(1).neq(0)  # bits: 2
    cloud_mask = qa_img.rightShift(3).bitwiseAnd(1).neq(0)   # bits: 3
    shadow_mask = qa_img.rightShift(4).bitwiseAnd(1).neq(0)  # bits: 4
    snow_mask = qa_img.rightShift(5).bitwiseAnd(1).neq(0)    # bits: 5
    clear_mask = qa_img.rightShift(6).bitwiseAnd(1).neq(0)   # bits: 6
    water_mask = qa_img.rightShift(7).bitwiseAnd(1).neq(0)   # bits: 7
    # cloud_conf = qa_img.rightShift(8).bitwiseAnd(3)          # bits: 8, 9
    # shadow_conf = qa_img.rightShift(10).bitwiseAnd(3)        # bits: 10, 11
    # snow_conf = qa_img.rightShift(12).bitwiseAnd(3)          # bits: 12, 13
    # cirrus_conf = qa_img.rightShift(14).bitwiseAnd(3)        # bits: 14, 15

    # Saturated pixels
    # Flag as saturated if any of the RGB bands are saturated
    #   or change .gt(0) to .gt(7) to flag if all RGB bands are saturated
    # Comment out rightShift line to flag if saturated in any band
    bitshift = ee.Dictionary({'LANDSAT_4': 0, 'LANDSAT_5': 0, 'LANDSAT_7': 0, 'LANDSAT_8': 1, 'LANDSAT_9': 1});
    saturated_mask = (
        landsat_img.select('QA_RADSAT')
        .rightShift(ee.Number(bitshift.get(ee.String(landsat_img.get('SPACECRAFT_ID'))))).bitwiseAnd(7)
        .gt(0)
    )
    
    # Old "Fmask" style image
    fmask_img = (
        qa_img.multiply(0)
        .where(landsat_img.select(['SR_B4']).mask().eq(0), 1)
        # .where(saturated_mask, 6)
        .where(water_mask, 5)
        .where(shadow_mask, 2)
        .where(snow_mask, 3)
        .where(cloud_mask.Or(dilate_mask).Or(cirrus_mask), 4)
        # .add(shadow_mask.multiply(2))
        # .add(snow_mask.multiply(3))
        # .add(cloud_mask.Or(dilate_mask).Or(cirrus_mask).multiply(4))
        # .add(cloud_mask.Or(dilate_mask).multiply(4))
        # .add(cloud_mask.And(cloud_conf).multiply(4))
        # .add(water_mask.multiply(5))
    )
    
    return fmask_img.updateMask(fmask_img.neq(0)).rename(['fmask'])


In [10]:
# Clean up the scene skip list file
for skip_path in [
    '../v2p1.csv',
    '../v2p1_eemetric.csv',
]:
    print(f'\n{skip_path}')
    with open(skip_path, 'r') as csv_f:
        scene_skip_lines = csv_f.readlines()
    scene_skip_header = scene_skip_lines.pop(0)
    # print(scene_skip_header)
    
    # Drop the comments and empty lines
    scene_skip_lines = [line.strip() for line in scene_skip_lines if line.strip() and line[0] != '#']
    
    # Sort by WRS2 tile then by date
    # scene_skip_lines = sorted(scene_skip_lines, key=lambda x:x.split(',')[0].split('_', 1)[1])

    # Sort by date then by tile
    scene_skip_lines = sorted(scene_skip_lines, key=lambda x:x.split(',')[0].split('_')[-1] + '_' + x.split(',')[0].split('_')[-2])

    
    # Identify duplicate scene IDs (as opposed to duplicate lines)
    # Note, this block is not removing any lines, just printing
    print('Duplicate Scene IDs:')
    
    if len({l.split(',')[0] for l in scene_skip_lines}) != len(scene_skip_lines):
        for item, count in collections.Counter([l.split(',')[0] for l in scene_skip_lines]).items():
            if count > 1:
                print(item)

    # Identify lines with no reason
    print('\nMissing reason Scene IDs:')
    for l in scene_skip_lines:
        if ',' not in l:
            print(l)
        elif l.split(',')[1].strip() == '':
            print(l)
        elif len(l.split(',')) > 2:
            print(l)
    
    # # Identify duplicate lines (not duplicate SCENE IDs)
    # if len({line for line in scene_skip_lines}) != len(scene_skip_lines):
    #     print('Duplicate Lines:')
    #     for item, count in collections.Counter(scene_skip_lines).items():
    #         if count > 1:
    #             print(item)
    # 
    #     # # Uncomment to have the tool remove duplicate lines
    #     # scene_remove_lines = []
    #     # for item, count in collections.Counter(scene_skip_lines).items():
    #     #     if count > 1:
    #     #         scene_remove_lines.append(item)
    #     #         # print(item)
    #      
    #     # # Does this only remove the first one?
    #     # if scene_remove_lines:
    #     #     print(f'Removing {len(scene_remove_lines)} duplicate lines in file')
    #     #     for line in scene_remove_lines:
    #     #         print(line)
    #     #         scene_skip_lines.remove(line)
    # 
    # # Then recheck for duplicate SCENE_IDs (but different notes or dates)
    # scenes = {line.split(',')[0] for line in scene_skip_lines}           
    # if len(scenes) != len(scene_skip_lines):
    #     print('Duplicate scene IDs still in file')
        
    print('\nWriting updated scene skip list CSV')
    with open(skip_path.replace('.csv', '_sorted.csv'), 'w') as csv_f:
        csv_f.write(scene_skip_header)
        for i, line in enumerate(scene_skip_lines):
            csv_f.write(line + '\n')

print('\nDone')


../v2p1.csv
Duplicate Scene IDs:

Missing reason Scene IDs:

Writing updated scene skip list CSV

../v2p1_eemetric.csv
Duplicate Scene IDs:

Missing reason Scene IDs:

Writing updated scene skip list CSV

Done


In [8]:
### Print scenes with low pixel count ratios (few unmasked pixels)
count_threshold_pct_min = 0
count_threshold_pct_max = 100
# count_threshold = 1

#start_year = 1984
#start_year = 1999
#start_year = 2016
start_year = 2024
end_year = 2024
years = list(range(start_year, end_year + 1))

print_count = 100
image_size = 900
# image_size = 1024

# Read in the scene skip list
scene_skip_url = '../v2p1.csv'
# scene_skip_url = 'https://raw.githubusercontent.com/cgmorton/scene-skip-list/main/v2p1.csv'
scene_skip_df = pd.read_csv(scene_skip_url)
scene_skip_df.set_index('SCENE_ID', inplace=True)
scene_skip_list = list(scene_skip_df.index.values)
print(f'Skip list images:       {len(scene_skip_list)}')

scene_cloudscore_url = '../v2p1_cloudscore.csv'
# scene_cloudscore_url = 'https://raw.githubusercontent.com/cgmorton/scene-skip-list/main/v2p1_cloudscore.csv'
scene_cloudscore_df = pd.read_csv(scene_cloudscore_url)
scene_cloudscore_df.set_index('SCENE_ID', inplace=True)
scene_cloudscore_list = list(scene_cloudscore_df.index.values)
print(f'Skip cloudscore images: {len(scene_cloudscore_list)}')

scene_eemetric_url = '../v2p1_eemetric.csv'
# scene_eemetric_url = 'https://raw.githubusercontent.com/cgmorton/scene-skip-list/main/v2p1_eemetric.csv'
scene_eemetric_df = pd.read_csv(scene_eemetric_url)
scene_eemetric_df.set_index('SCENE_ID', inplace=True)
scene_eemetric_list = list(scene_eemetric_df.index.values)
print(f'Skip EEMETRIC images:   {len(scene_eemetric_list)}')


print('Reading image stats CSV files')
stats_df_list = []
for wrs2_tile in wrs2_list:
    # if int(wrs2_tile[1:4]) not in range(10, 25):
    #     continue
        
    for year in range(start_year, end_year + 1):
        wrs2_stats_path = os.path.join(stats_ws, f'{year}', f'{wrs2_tile}_{year}.csv')
        if not os.path.isfile(wrs2_stats_path):
            # print(f'  {wrs2_tile}_{year} - Missing stats CSV, skipping')
            continue
        try:
            wrs2_stats_df = pd.read_csv(wrs2_stats_path)
        except Exception as e:
            print(f'  {wrs2_tile}_{year} - Error reading CSV, skipping')
            os.remove(wrs2_stats_path)
            continue
        wrs2_stats_df['DATE'] = wrs2_stats_df['SCENE_ID'].str.slice(12, 20)
        #wrs2_stats_df['WRS2'] = 'p' + wrs2_stats_df['SCENE_ID'].str.slice(5, 8) + 'r' + wrs2_stats_df['SCENE_ID'].str.slice(8, 11)
        stats_df_list.append(wrs2_stats_df)

stats_df = pd.concat(stats_df_list)

# Add the high CLOUD_COVER_LAND scenes to the skip list but don't remove from the dataframe
scene_skip_list.extend(stats_df[stats_df['CLOUD_COVER_LAND'] >= 71]['SCENE_ID'].values)

# Skip the Landsat 7 scenes in 2023
l7_2022_mask = (stats_df['DATE'].str.slice(0,4) >= '2022') & (stats_df['SCENE_ID'].str.slice(0,4) == 'LE07')
stats_df = stats_df[~l7_2022_mask]

# Compute the ratios
# stats_df['ACCA_COUNT_RATIO'] = stats_df['ACCA_PIXELS'] / stats_df['TOTAL_PIXELS']
stats_df['SNOW_COUNT_RATIO'] = stats_df['SNOW_PIXELS'] / stats_df['TOTAL_PIXELS']
# stats_df['SHADOW_COUNT_RATIO'] = stats_df['SHADOW_PIXELS'] / stats_df['TOTAL_PIXELS']
stats_df['MASKED_PIXELS'] = (
    stats_df['CLOUD_PIXELS']
    + stats_df['CIRRUS_PIXELS']
    + stats_df['DILATE_PIXELS']
    + stats_df['SHADOW_PIXELS']
    + stats_df['SNOW_PIXELS']
    # + stats_df['WATER_PIXELS']
    + stats_df['ACCA_PIXELS']
    # + stats_df['SATURATED_PIXELS']
)
stats_df['CLOUD_COUNT_RATIO'] = stats_df['MASKED_PIXELS'] / stats_df['TOTAL_PIXELS']
# stats_df['CLOUD_COUNT_RATIO'] = stats_df['UNMASKED_PIXELS'] / stats_df['TOTAL_PIXELS']

print(f'  {len(stats_df.count(axis=1))}')


# Applying skip list here so that main stats DF has all scenes
wrs2_stats_df = stats_df.copy()
wrs2_stats_df = wrs2_stats_df[wrs2_stats_df['SCENE_ID'].isin(scene_skip_list)]
wrs2_stats_df = wrs2_stats_df[~wrs2_stats_df['SCENE_ID'].isin(scene_eemetric_list)]
# wrs2_stats_df = wrs2_stats_df[~wrs2_stats_df['SCENE_ID'].isin(scene_cloudscore_list)]


print(scene_skip_df[~scene_skip_df['REASON'].str.contains('Snow')].index.values)
# Skip images that have Snow as a skip reason
wrs2_stats_df = wrs2_stats_df[wrs2_stats_df['SCENE_ID'].isin(scene_skip_df[~scene_skip_df['REASON'].str.contains('Snow')].index.values)]
# # Skip images that have Missing as a skip reason
# wrs2_stats_df = wrs2_stats_df[wrs2_stats_df['SCENE_ID'].isin(scene_skip_df[~scene_skip_df['REASON'].str.contains('Missing')].index.values)]
# # Skip images that only have Cloud as a skip reason
# wrs2_stats_df = wrs2_stats_df[wrs2_stats_df['SCENE_ID'].isin(scene_skip_df[~scene_skip_df['REASON']=='Cloud'].index.values)]

# Filter on the overall cloud count ratio
wrs2_stats_df = wrs2_stats_df[wrs2_stats_df['CLOUD_COUNT_RATIO'] < (count_threshold_pct_max / 100)]
wrs2_stats_df = wrs2_stats_df[wrs2_stats_df['CLOUD_COUNT_RATIO'] >= (count_threshold_pct_min / 100)]
wrs2_stats_df.sort_values('CLOUD_COUNT_RATIO', ascending=False, inplace=True)

print(f'  {len(wrs2_stats_df.count(axis=1))}')


new_skip_scenes = []
new_skip_count = 0

for i, row in wrs2_stats_df.iterrows():
    scene_id = row["SCENE_ID"].upper()
    
    wrs2_path = int(scene_id[5:8])
    wrs2_row = int(scene_id[8:11])
    #wrs2_path = int(wrs2[1:4])
    #wrs2_row = int(wrs2[5:8])
    wrs2_tgt = f'{wrs2_path:03d}{wrs2_row:03d}'
    wrs2_above = f'{wrs2_path:03d}{wrs2_row-1:03d}'
    wrs2_below = f'{wrs2_path:03d}{wrs2_row+1:03d}'

    above_scene_id = scene_id.upper().replace(wrs2_tgt, wrs2_above)
    above_stats_df = stats_df.loc[stats_df['SCENE_ID'] == above_scene_id]
    if len(above_stats_df):
        above_cloud_pct = float(above_stats_df.iloc[0]['CLOUD_COVER_LAND'])
    else:
        above_cloud_pct = None
        
    below_scene_id = scene_id.upper().replace(wrs2_tgt, wrs2_below)
    below_stats_df = stats_df.loc[stats_df['SCENE_ID'] == below_scene_id]
    if len(below_stats_df):
        below_cloud_pct = float(below_stats_df.iloc[0]['CLOUD_COVER_LAND'])
    else:
        below_cloud_pct = None

    # # Only show scenes that have above & below both skipped or None
    # if (((above_scene_id not in scene_skip_list) and (above_cloud_pct is not None)) or 
    #     ((below_scene_id not in scene_skip_list) and (below_cloud_pct is not None))):
    #     continue   

    # # Only show scenes that have eith the above & below scene skipped or None
    # if (((above_scene_id not in scene_skip_list) and (above_cloud_pct is not None)) and 
    #     ((below_scene_id not in scene_skip_list) and (below_cloud_pct is not None))):
    #     continue  
    
    landsat_type = scene_id.split('_')[0].upper()
    landsat_img = ee.Image(f'LANDSAT/{landsat_type}/C02/T1_L2/{scene_id}')
    landsat_region = landsat_img.geometry().bounds(1, 'EPSG:4326')
    landsat_sr_img = landsat_img.select(rgb_bands[landsat_type]).multiply([0.0000275]).add([-0.2])

    # Landsat true color image
    landsat_url = (
        landsat_sr_img.where(land_mask.unmask().eq(0), 0.25)
        .getThumbURL({'min': 0.0, 'max': 0.30, 'gamma': 1.25, 'region': landsat_region, 'dimensions': image_size})
    )

    # Landsat true color with Fmask
    fmask_url = (
        landsat_sr_img.where(land_mask.unmask().eq(0), 0.25).visualize(min=0, max=0.3, gamma=1.25)
        .blend(fmask(landsat_img).where(land_mask.unmask().eq(0), fmask_max).visualize(bands='fmask', min=0, max=fmask_max, palette=fmask_palette))
        .getThumbURL({'region': landsat_region, 'dimensions': image_size})
    )

    print('#'*80)
    print(
        f'  {scene_id}  {row["TOTAL_PIXELS"]:>10d}  {row["UNMASKED_PIXELS"]:>10d}'
        f'  ({row["CLOUD_COUNT_RATIO"]:>0.2f}) ({row["SNOW_COUNT_RATIO"]:>0.2f}) {row["CLOUD_COVER_LAND"]}'
        f'  {scene_id},{scene_skip_df.loc[scene_id, "REASON"]}'
    )
    ipyplot.plot_images([landsat_url, fmask_url], img_width=image_size)

    # Show the images above and below the target wrs2
    above_img = ee.Image(f'LANDSAT/{landsat_type}/C02/T1_L2/{above_scene_id}')
    above_region = above_img.geometry().bounds(1, 'EPSG:4326')
    above_sr_img = above_img.select(rgb_bands[landsat_type]).multiply([0.0000275]).add([-0.2])
    try:
        above_url = (
            above_sr_img.where(land_mask.unmask().eq(0), 0.25).visualize(min=0, max=0.3, gamma=1.25)
            .blend(fmask(above_img).where(land_mask.unmask().eq(0), fmask_max).visualize(bands='fmask', min=0, max=fmask_max, palette=fmask_palette))
            .getThumbURL({'region': above_region, 'dimensions': image_size})
        )
    except:
        above_url = None
        
    below_img = ee.Image(f'LANDSAT/{landsat_type}/C02/T1_L2/{below_scene_id}')
    below_region = below_img.geometry().bounds(1, 'EPSG:4326')
    below_sr_img = below_img.select(rgb_bands[landsat_type]).multiply([0.0000275]).add([-0.2])
    try:
        below_url = (
            below_sr_img.where(land_mask.unmask().eq(0), 0.25).visualize(min=0, max=0.3, gamma=1.25)
            .blend(fmask(below_img).where(land_mask.unmask().eq(0), fmask_max).visualize(bands='fmask', min=0, max=fmask_max, palette=fmask_palette))
            .getThumbURL({'region': below_region, 'dimensions': image_size})
        )
    except:
        below_url = None

    above_skipped = f' (skipped)' if above_scene_id in scene_skip_list else ''   
    below_skipped = f' (skipped)' if below_scene_id in scene_skip_list else ''
    
    if above_url and below_url:
        print(f'{below_scene_id} ({below_cloud_pct}){below_skipped}  {above_scene_id} ({above_cloud_pct}){above_skipped}')
        ipyplot.plot_images([below_url, above_url], img_width=image_size)
    elif above_url:
        print(f'{above_scene_id} ({above_cloud_pct}){above_skipped}')
        ipyplot.plot_images([above_url], img_width=image_size)
    elif below_url:
        print(f'{below_scene_id} ({below_cloud_pct}){below_skipped}')
        ipyplot.plot_images([below_url], img_width=image_size)

    new_skip_scenes.append(scene_id)
    new_skip_count += 1
    if new_skip_count >= print_count:
        break

if new_skip_scenes:
    for scene_id in new_skip_scenes:
        print(scene_id)

print('\nDone')


Skip list images:       47350
Skip cloudscore images: 180
Skip EEMETRIC images:   10847
Reading image stats CSV files
  15693
['LT05_034028_19840322' 'LT05_034036_19840322' 'LT05_018032_19840327' ...
 'LC09_028040_20241031' 'LC08_036034_20241031' 'LC09_044032_20241031']
  50
################################################################################
  LC08_018033_20240526    40213734     3804713  (0.91) (0.00) 61.1  LC08_018033_20240526,Cloud


LC08_018034_20240526 (91.31) (skipped)  LC08_018032_20240526 (53.9)


################################################################################
  LC09_044026_20240321    39919962     5582708  (0.86) (0.00) 69.55  LC09_044026_20240321,Cloud


LC09_044027_20240321 (47.97)


################################################################################
  LC09_021033_20240304    40127214     5883926  (0.85) (0.00) 70.44  LC09_021033_20240304,Cloud


...

################################################################################
  LC09_026040_20240526    32268419     8030372  (0.75) (0.00) 56.65  LC09_026040_20240526,Cloud


LC09_026039_20240526 (61.85) (skipped)


################################################################################
  LC09_028039_20240727    40436632    10734364  (0.73) (0.00) 50.86  LC09_028039_20240727,Cloud


LC09_028040_20240727 (54.78) (skipped)  LC09_028038_20240727 (30.69)


LC08_018033_20240526
LC09_044026_20240321
LC09_021033_20240304
LC08_023029_20240326
LC09_027035_20240618
LC08_041025_20240628
LC08_016040_20240309
LC09_037026_20240523
LC08_046026_20240530
LC09_016037_20240824
LC08_021035_20240515
LC09_020037_20240617
LC09_018035_20240619
LC09_021031_20240320
LC09_014027_20240506
LC09_019038_20240509
LC08_034028_20240526
LC08_020038_20240508
LC09_018036_20240603
LC08_045031_20240405
LC08_040026_20240520
LC09_015042_20240427
LC09_018039_20240619
LC09_020034_20240516
LC09_027036_20240618
LC09_024036_20240901
LC09_012028_20240812
LC09_038029_20240530
LC09_020036_20240430
LC09_012029_20240812
LC08_021029_20240328
LC08_026037_20240502
LC08_040028_20240504
LC09_044033_20240625
LC09_020034_20240430
LC09_027039_20240618
LC09_019039_20240509
LC08_030034_20240530
LC09_018034_20240619
LC09_025036_20240620
LC09_041025_20240519
LC08_019039_20240602
LC08_025039_20240527
LC09_027037_20240618
LC08_038035_20240404
LC09_023036_20240403
LC09_026039_20240526
LC09_018036_2